<a href="https://colab.research.google.com/github/AlexandrePardelinha/Computer-Science-Masters/blob/main/RFRegression_Roberto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV


In [ ]:
import time
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

**Importando as funções loadGeneralSettings, loadDatasetColumns do arquivo 'settings' e as funções selectDatasetColumns, add_pca, preprare_training, add_regioes do arquivo 'dataset'. Ambos os códigos dos arquivos 'settings' e 'dataset' estão na pasta 'utils'.
Deu erro (não reconhecendo o arquivo do diretório 'utils') e fiz de maneira diferente do original.** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
caminho_= '/content/drive/My Drive/SBDE-main'

In [ ]:
import sys
sys.path.insert(0,'/content/drive/My Drive/SBDE-main/utils')

In [ ]:
from settings import loadGeneralSettings, loadDatasetColumns
from dataset import selectDatasetColumns, add_pca, preprare_training, add_regioes

In [ ]:
#from utils.settings import loadGeneralSettings, loadDatasetColumns
#from utils.dataset import selectDatasetColumns, add_pca, preprare_training, add_regioes

In [ ]:
import pickle



*****************************************************************************************************

**Esse código está dentro da pasta 'utils' com nome do arquivo 'settings'. Mesmo ajustando o 'path' corretamente, não consegui importar o arquivo da pasta 'settings' (dando erro: No such file or diretory: 'settings.yaml'). Colei aqui o código e não deu erro. O arquivo 'settings.yaml' consta o caminho para a planilha do excel 'dataset'. 
dataset:
  path: /content/drive/My Drive/SBDE-main/data/dataset.xlsx**

In [ ]:
import yaml
def loadSettings(arquivo):
    with open(f'/content/drive/My Drive/SBDE-main/settings/{arquivo}.yaml') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

def loadGeneralSettings():
    return loadSettings('settings')

def loadDatasetColumns():
    return loadSettings('datasetColumns')

***************************************************************************************************************

**Esse código está dentro da pasta 'utils' com nome do arquivo 'dataset'. Mesmo ajustando o 'path' corretamente, não consegui importar os aquivos da pasta 'models', não reconhecendo os arquivos (os arquivos com formato '.sav'). Colei aqui o código e não deu erro.**

**Link do IBGE que separa em regiões: [{"id":1,"sigla":"N","nome":"Norte"},{"id":2,"sigla":"NE","nome":"Nordeste"},{"id":3,"sigla":"SE","nome":"Sudeste"},{"id":4,"sigla":"S","nome":"Sul"},{"id":5,"sigla":"CO","nome":"Centro-Oeste"}]**

In [ ]:
import pickle
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

def data_preprare(df_input, without_sr:bool=False):
    km = pickle.load(open('/content/drive/My Drive/SBDE-main/models/kmeansclusterization.sav', 'rb'))
    ec = pickle.load(open('/content/drive/My Drive/SBDE-main/models/encode_regiao.sav', 'rb'))

    df = df_input.drop(['pubtitle'], axis=1)

    if without_sr:
        df = remove_sr(df)

    df_reg = pd.DataFrame(ec.transform(df.regiao.values.reshape(-1,1)), columns=['centrooeste', 'nordeste', 'norte', 'sudeste', 'sul'])

    xpred = km.predict(df.drop(['ds', 'regiao'], axis=1))

    clusters = pd.DataFrame(xpred, columns=['cluster'])

    df = pd.concat([df, clusters, df_reg], axis=1).drop(['regiao'], axis=1)

    return (df.drop(['ds'], axis=1), df.ds)

def remove_sr(df_input):
    df =  df_input.drop(index=list(df_input[df_input.regiao=='S/R'].index))
    df = df.reset_index()
    return df.drop(['index'], axis=1)


def selectDatasetColumns(df, columns):
    columns_remove = df.columns.to_list()
    for i in df.columns.to_list():
        if i in columns:
            columns_remove.remove(i)
    return df.drop(columns=columns_remove)


def add_pca(datasetColumns)->dict:
    pca = pickle.load(open('/content/drive/My Drive/SBDE-main/models/pca.sav', 'rb'))
    for i in range(0, len(pca)):
        datasetColumns[pca[i][0]]['n_components_'+str(pca[i][1])] = i

    return datasetColumns


def add_regioes(datasetColumns)->dict:

    reg = pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/regioes')

    for nome in reg.nome.to_list():
        datasetColumns[nome] = {'all':['ds', 'sand', 'silt', 'clay', 'phh2o', 'sb', 'al', 'h', 'corg']}
        
    return datasetColumns

def preprare_training(datasetColumns, kcluster, key, dfs_training, dfs_test, df_pub_regioes):

    km = pickle.load(open('/content/drive/My Drive/SBDE-main/models/kmeansclusterization.sav', 'rb'))

    pca = pickle.load(open('/content/drive/My Drive/SBDE-main/models/pca.sav', 'rb'))

    reg = pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/regioes')

    if kcluster in reg.nome.to_list():
        dfs_training = pd.merge(dfs_training, df_pub_regioes, how='left', left_on="pubtitle", right_on="pub")
        dfs_training = dfs_training[dfs_training['regiao'] == str.upper(kcluster)]
        dfs_test = pd.merge(dfs_test, df_pub_regioes, how='left', left_on="pubtitle", right_on="pub")
        dfs_test = dfs_test[dfs_test['regiao'] == str.upper(kcluster)]

    if 'n_components' in key:
        columns = datasetColumns[kcluster]["all"]
    else:
        columns = datasetColumns[kcluster][key]

    dfs_training_adj = selectDatasetColumns(dfs_training, columns)
    dfs_training_x = dfs_training_adj.drop(columns=['ds'])
    dfs_training_y = dfs_training_adj.ds

    dfs_test_adj = selectDatasetColumns(dfs_test, columns)
    dfs_test_x = dfs_test_adj.drop(columns=['ds'])
    dfs_test_y = dfs_test_adj.ds

    if 'n_components' in key:
        dfs_training_x = pd.DataFrame(pca[datasetColumns[kcluster][key]][3].transform(dfs_training_x))
        dfs_test_x = pd.DataFrame(pca[datasetColumns[kcluster][key]][3].transform(dfs_test_x))

    elif 'cluster' in kcluster:

        idx = int(kcluster.replace('cluster', ''))

        X = selectDatasetColumns(dfs_training, datasetColumns['all']['all']).drop(columns=['ds'])
        xpred = km.predict(X)
        clusters = pd.DataFrame(xpred, columns=['cluster'])

        dfs_training_x_cluster = pd.concat([dfs_training_x, clusters], axis=1)
        dfs_training_x_cluster_idx = dfs_training_x_cluster[dfs_training_x_cluster.cluster == idx].index.tolist()
        dfs_training_x = dfs_training_x.loc[dfs_training_x_cluster_idx]
        dfs_training_y = dfs_training_y.loc[dfs_training_x_cluster_idx]

        X = selectDatasetColumns(dfs_test, datasetColumns['all']['all']).drop(columns=['ds'])
       
        xpred = km.predict(X)
        clusters = pd.DataFrame(xpred, columns=['cluster'])

        dfs_test_x_cluster = pd.concat([dfs_test_x, clusters], axis=1)
        dfs_test_x_cluster_idx = dfs_test_x_cluster[dfs_test_x_cluster.cluster == idx].index.tolist()
        dfs_test_x = dfs_test_x.loc[dfs_test_x_cluster_idx]
        dfs_test_y = dfs_test_y.loc[dfs_test_x_cluster_idx]

    return (dfs_training_x, dfs_training_y, dfs_test_x, dfs_test_y)

**************************************************************************************************************************

**Nessa fase que estava dando o erro: 'No such file or directory: 'models/pca.sav', mesmo ajustando os 'path' do arquivo 'dataset' da pasta 'utils' (o código anterior a esse). Depois que colei diretamente aqui, não deu mais esse erro.**

In [ ]:
settings = loadGeneralSettings()
datasetColumns = loadDatasetColumns()

In [ ]:
datasetColumns

{'all': {'all': ['ds',
   'sand',
   'silt',
   'clay',
   'phh2o',
   'sb',
   'al',
   'h',
   'corg'],
  'colinear1': ['ds', 'silt', 'clay', 'phh2o', 'sb', 'al', 'h', 'corg'],
  'colinear2': ['ds', 'sand', 'silt', 'phh2o', 'sb', 'al', 'h', 'corg'],
  'corrleq10': ['ds', 'sand', 'silt', 'clay', 'phh2o', 'h', 'corg'],
  'corrleq20': ['ds', 'sand', 'clay', 'h', 'corg']},
 'cluster0': {'all': ['ds',
   'sand',
   'silt',
   'clay',
   'phh2o',
   'sb',
   'al',
   'h',
   'corg'],
  'corrleq10': ['ds', 'sand', 'clay', 'phh2o', 'sb', 'al', 'h', 'corg'],
  'corrleq20': ['ds', 'clay', 'sb', 'h', 'corg']},
 'cluster1': {'all': ['ds',
   'sand',
   'silt',
   'clay',
   'phh2o',
   'sb',
   'al',
   'h',
   'corg'],
  'corrleq10': ['ds', 'sand', 'silt', 'clay', 'phh2o', 'sb', 'h', 'corg'],
  'corrleq20': ['ds', 'silt', 'phh2o', 'h', 'corg']},
 'cluster2': {'all': ['ds',
   'sand',
   'silt',
   'clay',
   'phh2o',
   'sb',
   'al',
   'h',
   'corg'],
  'corrleq10': ['ds', 'clay', 'phh2o', '

**Estava dando erro: 'path_regioes' e não consegui descobrir o que é isso. Abri o arquivo 'dataset' e tem uma coluna chamada 'regiao' (coluna 12). Usei essa coluna para o pub_reg. O problema é que existe essa coluna 'regiao' nas 3 abas (o arquivo 'dataset' tem 3 abas: 'training', 'validation' e 'test') e não sei se pegou nas 3 abas ou somente da primeira.**

In [ ]:
dfs_training = pd.read_excel(f'{caminho_}/data/Treinamento-validacao-solos-minerais.xlsx', 
                    sheet_name='treinamento')

dfs_test = pd.read_excel(f'{caminho_}/data/Treinamento-validacao-solos-minerais.xlsx', 
                    sheet_name='validacao')

pub_reg = pd.read_excel(f'{caminho_}/data/pub.xls')

#pub_reg = pd.read_excel(settings['dataset']['path'],
                        #usecols=[12])

**Está dando erro no 'all'. O Roberto me falou que o 'all' são todas as variáveis do 'dataset' mas não sei como corrigir esse erro.**

In [ ]:
datasetColumns = add_pca(datasetColumns)
datasetColumns = add_regioes(datasetColumns)
reg = pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/regioes')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
param_grid = {
    "max_depth": list(range(1, 150, 2)),
    "n_estimators": list(range(1, 100, 2)),
    #"max_features": ['auto', 'sqrt', 'log2'],
    #"bootstrap": [True, False]
}

**Não precisou usar o 'test_size' pois no 'dataset' já estão separados os treinos, testes e validação. Roberto disse que é mais fácil de produzir. Eu, particularmente, prefiro usar o 'test_size' por ser menos trabalhoso.**

**y = ds (densidade do solo)**


**x = restante das variáveis**

In [ ]:
results = []

for kcluster in datasetColumns:
    for key in datasetColumns[kcluster]:

        if kcluster not in reg.nome.to_list():
            continue

        dfs_training_x, dfs_training_y, dfs_test_x, dfs_test_y = preprare_training(datasetColumns, kcluster, key, dfs_training, dfs_test, pub_reg)

        ini = time.time()
        sh = HalvingGridSearchCV(
            RandomForestRegressor(criterion="squared_error"),
            param_grid,
            cv=5,
            n_jobs=6,
            scoring='neg_mean_squared_error',
            verbose=2

        ).fit(
            X=dfs_training_x, 
            y=dfs_training_y
            )
        t = time.time() - ini

        predict = sh.best_estimator_.predict(dfs_test_x)

        #pickle.dump(sh.best_estimator_, open(f'{caminho_}/models/rf/' + kcluster + '_' + key + '.sav', 'wb'))
        pickle.dump(sh.best_estimator_, open(f'{caminho_}/models/rf/' + kcluster + '_' + key + '.sav', 'wb'))

        line = ['RF', kcluster, key, t, mean_squared_error(dfs_test_y, predict), mean_absolute_error(dfs_test_y, predict), mean_absolute_percentage_error(dfs_test_y, predict), str(sh.best_estimator_)]

        results.append(line)

----------
iter: 1
n_candidates: 1250
n_resources: 30
Fitting 5 folds for each of 1250 candidates, totalling 6250 fits
----------
iter: 2
n_candidates: 417
n_resources: 90
Fitting 5 folds for each of 417 candidates, totalling 2085 fits
----------
iter: 3
n_candidates: 139
n_resources: 270
Fitting 5 folds for each of 139 candidates, totalling 695 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


n_iterations: 4
n_required_iterations: 8
n_possible_iterations: 4
min_resources_: 10
max_resources_: 340
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3750
n_resources: 10
Fitting 5 folds for each of 3750 candidates, totalling 18750 fits
----------
iter: 1
n_candidates: 1250
n_resources: 30
Fitting 5 folds for each of 1250 candidates, totalling 6250 fits
----------
iter: 2
n_candidates: 417
n_resources: 90
Fitting 5 folds for each of 417 candidates, totalling 2085 fits
----------
iter: 3
n_candidates: 139
n_resources: 270
Fitting 5 folds for each of 139 candidates, totalling 695 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


n_iterations: 5
n_required_iterations: 8
n_possible_iterations: 5
min_resources_: 10
max_resources_: 1124
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3750
n_resources: 10
Fitting 5 folds for each of 3750 candidates, totalling 18750 fits
----------
iter: 1
n_candidates: 1250
n_resources: 30
Fitting 5 folds for each of 1250 candidates, totalling 6250 fits
----------
iter: 2
n_candidates: 417
n_resources: 90
Fitting 5 folds for each of 417 candidates, totalling 2085 fits
----------
iter: 3
n_candidates: 139
n_resources: 270
Fitting 5 folds for each of 139 candidates, totalling 695 fits
----------
iter: 4
n_candidates: 47
n_resources: 810
Fitting 5 folds for each of 47 candidates, totalling 235 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


n_iterations: 3
n_required_iterations: 8
n_possible_iterations: 3
min_resources_: 10
max_resources_: 176
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3750
n_resources: 10
Fitting 5 folds for each of 3750 candidates, totalling 18750 fits
----------
iter: 1
n_candidates: 1250
n_resources: 30
Fitting 5 folds for each of 1250 candidates, totalling 6250 fits
----------
iter: 2
n_candidates: 417
n_resources: 90
Fitting 5 folds for each of 417 candidates, totalling 2085 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


n_iterations: 3
n_required_iterations: 8
n_possible_iterations: 3
min_resources_: 10
max_resources_: 201
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3750
n_resources: 10
Fitting 5 folds for each of 3750 candidates, totalling 18750 fits
----------
iter: 1
n_candidates: 1250
n_resources: 30
Fitting 5 folds for each of 1250 candidates, totalling 6250 fits
----------
iter: 2
n_candidates: 417
n_resources: 90
Fitting 5 folds for each of 417 candidates, totalling 2085 fits


**'time': tempo de execução em segundos.**

In [ ]:
dfr = pd.DataFrame(results, columns=['Method', 'Cluster', 'DatasetColumns', 'time', 'mse', 'mae', 'mape', 'coef'])

**Provavelmente terei que mudar o path do rf2.pkl.**

In [ ]:
dfr.to_pickle(f'{caminho_}/results/rf2.pkl')

**Resultado**

In [ ]:
print(dfr)

  Method       Cluster DatasetColumns         time       mse       mae  \
0     RF         Norte            all  2399.818646  0.013278  0.089942   
1     RF      Nordeste            all  2482.483153  0.027928  0.132922   
2     RF       Sudeste            all  2321.241932  0.015572  0.100720   
3     RF           Sul            all  2146.935604  0.013886  0.092801   
4     RF  Centro-Oeste            all  2199.935708  0.007049  0.068339   

       mape                                               coef  
0  0.072975  RandomForestRegressor(max_depth=23, n_estimato...  
1  0.092523  RandomForestRegressor(max_depth=3, n_estimator...  
2  0.079634  RandomForestRegressor(max_depth=77, n_estimato...  
3  0.083011  RandomForestRegressor(max_depth=131, n_estimat...  
4  0.063722  RandomForestRegressor(max_depth=91, n_estimato...  
